# Write a grid file

Either from scratch given a certain number of binaries, or based off the output from an already existing grid

In [34]:
import numpy as np
import random
import math
import h5py

N_binaries = int(1e6)

# Grid file containing only metallicities

For your fiducial run, you just want to run COMPAS with fiducial flags, but drawing metallicities that are distributed uniformly in log


In [33]:
#################################################################
## 
##   Write a grid file to use
##
#################################################################
def write_grid_file(N = 10, grid_file_name = 'BSE_grid.txt' ):
    """
    N is number of lines
    """

    # Define the minimum and maximum values for Z
    Z_min = 1e-4
    Z_max = 0.03

    # Set a random seed for reproducibility
    random.seed(42)

    # Open the output file for writing
    with open(grid_file_name, 'w') as f:
        # Write each Z value to a separate line in the file, with a random seed integer
        for i in range(N):
            Z = 10 ** random.uniform(math.log10(Z_min), math.log10(Z_max))
            f.write('--random-seed {} --metallicity {}\n'.format(i, Z))


write_grid_file(N = N_binaries, grid_file_name = './masterfolder/BSE_grid.txt' )

# Write a grid file based on the system params of an existing simulation

By reading the 'BSE_System_Parameters', make a grid file to run with compas 
at least containing the parameters as set in Grid_dict 

The grid will be quite large (~1 Gb for 10^6 binaries) so this takes about a minute


In [40]:

# Define the path to the HDF5 file
root_out_dir = "/n/holystore01/LABS/hernquist_lab/Users/lvanson/CompasOutput/v02.35.02/FiducialN1e6/"
filename     = "MainRun/COMPAS_Output.h5"
filepath     = root_out_dir + filename

## WARNING ##
"""
BSE_System_Parameters will only contain all the kick information if the simulation was run 
using a grid, and with the flag --add-options-to-sysparms = 'GRID'
"""
# Define the dictionary of keys and command line arguments
Grid_dict = {'SEED':'--random-seed', 'Mass@ZAMS(1)': '--initial-mass-1', 'Mass@ZAMS(2)':'--initial-mass-2',
             'SemiMajorAxis@ZAMS':'--semi-major-axis','Metallicity@ZAMS(1)':'--metallicity',\
             'Kick_Magnitude_Random(1)':'--kick-magnitude-random-1', 'Kick_Phi(1)':'--kick-phi-1',\
             'Kick_Theta(1)':'--kick-theta-1', 'Kick_Mean_Anomaly(1)':'--kick-mean-anomaly-1',\
             'Kick_Magnitude_Random(2)':'--kick-magnitude-random-2', 'Kick_Phi(1)':'--kick-phi-2',\
             'Kick_Theta(2)':'--kick-theta-2', 'Kick_Mean_Anomaly(2)':'--kick-mean-anomaly-2'}
             
# Open the HDF5 file in read mode
with h5py.File(filepath, 'r') as data:

    # Get the number of rows in the data
    n_rows = len(data['BSE_System_Parameters']['SEED'])
    
    # Initialize a NumPy array to hold the data
    data_array = np.zeros((n_rows, len(Grid_dict)), dtype=np.float64)
        
    # Loop over the keys in the Grid_dict dictionary
    for i, key in enumerate(Grid_dict.keys()):
        
        # Get the data from the HDF5 file and store it in the data array
        data_key = 'BSE_System_Parameters/' + key
        if key == 'SEED':  # Convert the SEED value to an integer
            data_array[:, i] = data[data_key][()].astype(int)
        else:
            data_array[:, i] = data[data_key][()]  
    
    # Format the data as command line arguments
    data_list = []
    for i in range(n_rows):
        row_data = data_array[i]
        row_list = ['{} {:d}'.format(cmd_arg, int(value)) if cmd_arg == '--random-seed' else '{} {}'.format(cmd_arg, value) for cmd_arg, value in zip(Grid_dict.values(), row_data)]
        data_line = ' '.join(row_list) + '\n'
        data_list.append(data_line)


    # Write the data to the output file
    with open('BSE_grid_N1e6.txt', 'w') as outfile:
        
        # Write the data to the output file
        outfile.writelines(data_list)


# Drawing Kicks

We need to draw kick magnitudes and orientations

### kick magnitudes:
By default, COMPAS assumes for BHs that kick magnitudes are drawn from a maxwellian (e.g., Hobbs et al., 2005),  with a sigma = 265.0 km/s

**--kick-magnitude-sigma-CCSN-BH 265.0** <br>
**--kick-magnitude-distribution  MAXWELLIAN**

_However_  we don't actually need to draw this ourselves! the parameters **--kick-magnitude-random-1**, and     **--kick-magnitude-random-2**, will cause a unique draw from this magnitude distribution for you!
This means that we just need to randomly drawn uniformly from `[0.0, 1.0)`


### Kick orientations:
By default, COMPAS draws natal kick directions from an ISOTROPIC distribution. 
i.e.,

**--kick-direction ISOTROPIC** 

This means that we want to set 
    1. --kick-theta-1:   angle between the orbital plane and the 'z' axis of the supernova vector for the primary star should it undergo a supernova event [radians]
    
        theta = acos(1.0 - (2.0 * RAND->Random())) - M_PI_2

    
    2. --kick-phi-1:   angle between 'x' and 'y', both in the orbital plane of the supernova vector, for the primary star should it undergo a supernova event [radians]

        phi   = RAND->Random() * _2_PI
        
    3. --kick-mean-anomaly-1: The mean anomaly at the instant of the supernova for the primary star of a binary system when evolving in BSE mode, should it undergo a supernova event. |br| Must be a floating-point number in the range :math:`[0.0, 2\pi)`. 
        
theta = np.arccos(1.0 - (2.0 * np.random.rand())) - np.pi/2
phi = np.random.rand() * 2 * np.pi



In [ ]:
############################################
##  WRITE SEEDS TO GRID RUN FILE
############################################
def write_BSE_grid(SYS, seed_list, save_loc = './',  BSE_grid_name = "BSE_grid.txt"):
    """ Function to write a BSE grid file for your systems of initerest. 
    SYS       = astropy table version of your SystemParameters
    seed_list = the Seeds of interest
    
    """
    Grid_keys = ['SEED','Mass@ZAMS(1)', 'Mass@ZAMS(2)', 'SemiMajorAxis@ZAMS','Metallicity@ZAMS(1)',\
            'SN_Kick_Magnitude_Random_Number(1)', 'SN_Kick_Magnitude_Random_Number(2)',\
#            'SN_Kick_Theta(1)', 'SN_Kick_Theta(2)', 'SN_Kick_Phi(1)', 'SN_Kick_Phi(2)',\
#            'SN_Kick_Mean_Anomaly(1)', 'SN_Kick_Mean_Anomaly(2)']
                ]
    Grid_dict = {'SEED':'--random-seed', 'Mass@ZAMS(1)': '--initial-mass-1', 'Mass@ZAMS(2)':'--initial-mass-2',\
                 'SemiMajorAxis@ZAMS':'--semi-major-axis','Metallicity@ZAMS(1)':'--metallicity',\
            'SN_Kick_Magnitude_Random_Number(1)':'--kick-magnitude-random-1', 'SN_Kick_Magnitude_Random_Number(2)':'--kick-magnitude-random-2',\
#            'SN_Kick_Theta(1)':'--kick-theta-1', 'SN_Kick_Theta(2)':'--kick-theta-2', 'SN_Kick_Phi(1)':'--kick-phi-1', 'SN_Kick_Phi(2)':'--kick-phi-2',\
#            'SN_Kick_Mean_Anomaly(1)':'--kick-mean-anomaly-1', 'SN_Kick_Mean_Anomaly(2)':'--kick-mean-anomaly-2'}
                }
    ##################################
    print('seed_list', seed_list)
    SYS_DCO_seeds_bool = np.in1d(SYS['SEED'][()], seed_list) #Bool to point SYS to DCO
    ##################################
    # Make a reduced table that only consists of your systems of interest
    Sub_SYS = SYS[SYS_DCO_seeds_bool]
    # Make a new file 
#     outF = open(save_loc + str(sys['SEED'])+BSE_grid_name , "w")
    outF = open(save_loc + BSE_grid_name , "w")
    # Loop over every system in seed_list
    for sys in Sub_SYS:
        print(save_loc + str(sys['SEED'])+BSE_grid_name)
        grid_line = ''
        # Append all the necessary keys
        for key in Grid_keys:
            grid_line += ' '+Grid_dict[key]+' '+str(sys[key])
        #print('grid_line',grid_line)
        outF.write(grid_line)
        outF.write("\n")
    outF.close()
